In [2]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [3]:
from tensorflow import keras

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout

In [5]:
from sklearn import metrics


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
lemmatizer = WordNetLemmatizer()

In [8]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [10]:
sample_submission = pd.read_csv("sample_submission.csv")

# Limpeza dos dados de entrada

In [11]:
sample_tweets_train=train_df['text']
sample_tweets_test=test_df['text']

In [12]:
def limpeza(tweets):
    tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
    tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
    tweets = tweets.str.replace(r'[0-9]','')
    tweets = tweets.replace("\\x89Û_",'')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÓ','')
    tweets = tweets.replace('\x89ÛÒ','')
    tweets = tweets.replace('\x89Û','')
    tweets = tweets.replace('\x89Û÷','')
    tweets = tweets.replace('\x89ûï','')
    #retirada de símbolos não alfanuméticos 
    tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")
    tweets = tweets.str.lower()
    return tweets

In [13]:
sample_tweets_train = limpeza(sample_tweets_train)
sample_tweets_test = limpeza(sample_tweets_test)

C:\Users\yfrom\AppData\Local\Temp\ipykernel_10664\1236119256.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_10664\1236119256.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_10664\1236119256.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'[0-9]','')
C:\Users\yfrom\AppData\Local\Temp\ipykernel_10664\1236119256.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")


In [14]:
stop_words = set(stopwords.words("english"))
def lematizar(tweets):
    sample_tweets= []
    for tweet in tweets:
        filtered_list = []
        words_lemmatized = []
        words = []
        words = word_tokenize(tweet) 
        words_lemmatized = [lemmatizer.lemmatize(word) for word in words] 
        for word in words_lemmatized:
            if word not in stop_words:
                filtered_list.append(word)
        sample_tweets.append(filtered_list)
    return sample_tweets 

In [15]:
sample_tweets_train = lematizar(sample_tweets_train)
sample_tweets_test = lematizar(sample_tweets_test)

In [16]:
train_df['words']= sample_tweets_train
test_df['words']= sample_tweets_test
test_df = test_df.drop(columns=['location', 'keyword','id','text'])
train_df = train_df.drop(columns=['location', 'keyword','id','text'])
train_df['text'] = train_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))
test_df['text'] = test_df['words'].apply(lambda x: ' '.join([str(elem) for elem in x]))

In [17]:
train_df.head()

,target,words,text
0,1,"[deed, reason, earthquake, may, allah, forgive...",deed reason earthquake may allah forgive u
1,1,"[forest, fire, near, la, ronge, sask, canada]",forest fire near la ronge sask canada
2,1,"[resident, asked, 'shelter, place, ', notified...",resident asked 'shelter place ' notified offic...
3,1,"[people, receive, wildfire, evacuation, order,...",people receive wildfire evacuation order calif...
4,1,"[got, sent, photo, ruby, alaska, smoke, wildfi...",got sent photo ruby alaska smoke wildfire pour...


In [18]:
test_df.head()

,words,text
0,"[happened, terrible, car, crash]",happened terrible car crash
1,"[heard, earthquake, different, city, stay, saf...",heard earthquake different city stay safe ever...
2,"[forest, fire, spot, pond, goose, fleeing, acr...",forest fire spot pond goose fleeing across str...
3,"[apocalypse, lighting, spokane, wildfire]",apocalypse lighting spokane wildfire
4,"[typhoon, soudelor, kill, china, taiwan]",typhoon soudelor kill china taiwan


# Tokenizar: Pad_sequence()

In [19]:
tfidf_vectorizer = TfidfVectorizer()
count_treino_tfidf = tfidf_vectorizer.fit_transform(train_df['text'])
count_teste_tfidf = tfidf_vectorizer.transform(test_df['text'])

In [36]:
max_features=5000
tokenizer=Tokenizer(num_words=max_features,split=' ')
l=50
tokenizer.fit_on_texts(train_df['text'])
tokenizer.fit_on_texts(test_df['text'])
X_train = tokenizer.texts_to_sequences(train_df['text'])
X_test = tokenizer.texts_to_sequences(test_df['text'])
X_test = pad_sequences(X_test, maxlen =l)
X_train = pad_sequences(X_train, maxlen =l)

In [21]:
y_train = train_df['target']

In [32]:
test_df.shape

(3263, 2)

In [33]:
train_df.shape

(7613, 3)

In [37]:
X_test.shape


(3263, 50)

In [38]:
X_train.shape

(7613, 50)

# LSTM

In [39]:
embed_dim = 100
lstm_out = 100
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, return_sequences=True,recurrent_dropout=0.4))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out,dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
adam = optimizers.Adam(learning_rate=2e-3)
model.compile(loss = 'binary_crossentropy', optimizer=adam ,metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 100)           500000    
                                                                 
 dropout_3 (Dropout)         (None, 50, 100)           0         
                                                                 
 lstm_2 (LSTM)               (None, 50, 100)           80400     
                                                                 
 dropout_4 (Dropout)         (None, 50, 100)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                

In [40]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [41]:
model.fit(X_train,y_train, epochs = 10,validation_split = 0.2 ,callbacks=[es_callback], batch_size=32)

Epoch 1/10
191/191 [==============================] - 66s 315ms/step - loss: 0.5236 - accuracy: 0.7452 - val_loss: 0.4590 - val_accuracy: 0.7991
Epoch 2/10
191/191 [==============================] - 55s 287ms/step - loss: 0.3610 - accuracy: 0.8465 - val_loss: 0.4960 - val_accuracy: 0.7590
Epoch 3/10
191/191 [==============================] - 54s 285ms/step - loss: 0.2909 - accuracy: 0.8865 - val_loss: 0.5336 - val_accuracy: 0.7505
Epoch 4/10
191/191 [==============================] - 54s 284ms/step - loss: 0.2412 - accuracy: 0.9072 - val_loss: 0.6261 - val_accuracy: 0.7538


In [42]:
y_pred = model.predict(X_test).round()

102/102 [==============================] - 6s 49ms/step


In [44]:
y_pred.shape

(3263, 1)

In [46]:
sample_submission["target"]=y_pred

# Avaliação

In [43]:
train_accuracy = round(metrics.accuracy_score(y_train,model.predict(X_train).round())*100)
train_accuracy

238/238 [==============================] - 12s 48ms/step


90

# Submissão 

In [48]:
sample_submission.to_csv("submission.csv", index=False)